# Amazon Fine Food Reviews Analysis
Data Source: https://www.kaggle.com/snap/amazon-fine-food-reviews 

EDA: https://nycdatascience.com/blog/student-works/amazon-fine-foods-visualization/

The Amazon Fine Food Reviews dataset consists of reviews of fine foods from Amazon.

Number of reviews: 568,454

Number of users: 256,059

Number of products: 74,258

Timespan: Oct 1999 - Oct 2012

Number of Attributes/Columns in data: 10

Attribute Information:

Id

ProductId - unique identifier for the product

UserId - unqiue identifier for the user

ProfileName

HelpfulnessNumerator - number of users who found the review helpful

HelpfulnessDenominator - number of users who indicated whether they found the review helpful or not

Score - rating between 1 and 5

Time - timestamp for the review

Summary - brief summary of the review

Text - text of the review

Objective:
Given a review, determine whether the review is positive (Rating of 4 or 5) or negative (rating of 1 or 2).


[Q] How to determine if a review is positive or negative?

[Ans] We could use the Score/Rating. A rating of 4 or 5 could be cosnidered a positive review. A review of 1 or 2 could be considered negative. A review of 3 is nuetral and ignored. This is an approximate and proxy way of determining the polarity (positivity/negativity) of a review.

# [1] Reading data
The dataset is available in two forms

1) CSV file

2) SQLite Database

In order to load the data, We have used the SQLITE dataset as it easier to query the data and visualise the data efficiently. 

Here as we only want to get the global sentiment of the recommendations (positive or negative), we will purposefully ignore all Scores equal to 3. If the score id above 3, then the recommendation wil be set to "positive". Otherwise, it will be set to "negative".

In [ ]:
import warnings
warnings.filterwarnings("ignore")


from sklearn.manifold import TSNE
from sklearn import datasets


import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

import re #regular expression


import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

from tqdm import tqdm
import os

print(os.listdir("../input"))

In [ ]:
# using the SQLite Table to read data.
con = sqlite3.connect('../input/database.sqlite')

# not taking into consideration those reviews with Score=3
filtered_data = pd.read_sql_query(""" SELECT * FROM reviews WHERE Score != 3 LIMIT 5000""", con)
filtered_data.columns
filtered_data.head()

# Give reviews with Score>3 a positive rating, and reviews with a score<3 a negative rating.
def partition(x):
    if x < 3:
        return 0
    return 1

#changing reviews with score less than 3 to be positive and vice-versa
actualScore = filtered_data['Score']
positiveNegative = actualScore.map(partition) 
filtered_data['Score'] = positiveNegative
print("Number of data points in our data", filtered_data.shape)
filtered_data.head(3)

In [ ]:
display = pd.read_sql_query("""
SELECT UserId, ProductId, ProfileName, Time, Score, Text, COUNT(*)
FROM Reviews
GROUP BY UserId
HAVING COUNT(*)>1
""", con)

In [ ]:
print(display.shape)
display.head()

In [ ]:
display[display['UserId']=='AZY10LLTJ71NX']

In [ ]:
display['COUNT(*)'].sum()

# [2] Data Cleaning: Deduplication

It is observed (as shown in the table below) that the reviews data had many duplicate entries. Hence it was necessary to remove duplicates in order to get unbiased results for the analysis of the data.

In [ ]:
display= pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3 AND UserId="AR5J8UI46CURR"
ORDER BY ProductID
""", con)
display.head()

As can be seen above the same user has multiple reviews of the with the same values for HelpfulnessNumerator, HelpfulnessDenominator, Score, Time, Summary and Text  and on doing analysis it was found that <br>

ProductId=B000HDOPZG was Loacker Quadratini Vanilla Wafer Cookies, 8.82-Ounce Packages (Pack of 8)<br>
<br> 
ProductId=B000HDL1RQ was Loacker Quadratini Lemon Wafer Cookies, 8.82-Ounce Packages (Pack of 8) and so on<br>

It was inferred after analysis that reviews with same parameters other than ProductId belonged to the same product just having different flavour or quantity. Hence in order to reduce redundancy it was decided to eliminate the rows having same parameters.<br>

The method used for the same was that we first sort the data according to ProductId and then just keep the first similar product review and delelte the others. for eg. in the above just the review for ProductId=B000HDL1RQ remains. This method ensures that there is only one representative for each product and deduplication without sorting would lead to possibility of different representatives still existing for the same product.

In [ ]:
#Sorting data according to ProductId in ascending order
sorted_data=filtered_data.sort_values('ProductId', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

In [ ]:
#Deduplication of entries
final=sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"}, keep='first', inplace=False)
final.shape

In [ ]:
#Checking to see how much % of data still remains
(final['Id'].size*1.0)/(filtered_data['Id'].size*1.0)*100

Observation:- It was also seen that in two rows given below the value of HelpfulnessNumerator is greater than HelpfulnessDenominator which is not practically possible hence these two rows too are removed from calcualtions

In [ ]:
display= pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3 AND Id=44737 OR Id=64422
ORDER BY ProductID
""", con)

display.head()

In [ ]:
final=final[final.HelpfulnessNumerator<=final.HelpfulnessDenominator]

In [ ]:
#Before starting the next phase of preprocessing lets see the number of entries left
print(final.shape)

#How many positive and negative reviews are present in our dataset?
final['Score'].value_counts()

# [3] Text Preprocessing

Now that we have finished deduplication our data requires some preprocessing before we go on further with analysis and making the prediction model.

Hence in the Preprocessing phase we do the following in the order below:-

1) Begin by removing the html tags
<br>2) Remove any punctuations or limited set of special characters like , or . or # etc.
<br>3) Check if the word is made up of english letters and is not alpha-numeric
<br>4) Check to see if the length of the word is greater than 2 (as it was researched that there is no adjective in 2-letters)
<br>5) Convert the word to lowercase
<br>6) Remove Stopwords
<br>7) Finally Snowball Stemming the word (it was obsereved to be better than Porter Stemming)

After which we collect the words used to describe positive and negative reviews

In [ ]:
# printing some random reviews
sent_0 = final['Text'].values[0]
print(sent_0)
print("="*50)

sent_1000 = final['Text'].values[1000]
print(sent_1000)
print("="*50)

sent_1500 = final['Text'].values[1500]
print(sent_1500)
print("="*50)

sent_4900 = final['Text'].values[4900]
print(sent_4900)
print("="*50)

In [ ]:
# remove urls from text python: https://stackoverflow.com/a/40823105/4084039
sent_0 = re.sub(r"http\S+", "", sent_0)
sent_1000 = re.sub(r"http\S+", "", sent_1000)
sent_150 = re.sub(r"http\S+", "", sent_1500)
sent_4900 = re.sub(r"http\S+", "", sent_4900)

print(sent_0)

In [ ]:
# https://stackoverflow.com/questions/16206380/python-beautifulsoup-how-to-remove-all-tags-from-an-element
from bs4 import BeautifulSoup

soup = BeautifulSoup(sent_0, 'lxml')
text = soup.get_text()
print(text)
print("="*50)

soup = BeautifulSoup(sent_1000, 'lxml')
text = soup.get_text()
print(text)
print("="*50)

soup = BeautifulSoup(sent_1500, 'lxml')
text = soup.get_text()
print(text)
print("="*50)

soup = BeautifulSoup(sent_4900, 'lxml')
text = soup.get_text()
print(text)

In [ ]:
# https://stackoverflow.com/a/47091490/4084039
import re

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [ ]:
sent_1500 = decontracted(sent_1500)
print(sent_1500)
print("="*50)

In [ ]:
#remove words with numbers python: https://stackoverflow.com/a/18082370/4084039
sent_0 = re.sub("\S*\d\S*", "", sent_0).strip()
print(sent_0)

In [ ]:
#remove spacial character: https://stackoverflow.com/a/5843547/4084039
sent_1500 = re.sub('[^A-Za-z0-9]+', ' ', sent_1500)
print(sent_1500)

In [ ]:
# https://gist.github.com/sebleier/554280
# we are removing the words from the stop words list: 'no', 'nor', 'not'
# <br /><br /> ==> after the above steps, we are getting "br br"
# we are including them into stop words list
# instead of <br /> if we have <br/> these tags would have revmoved in the 1st step

stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

In [ ]:
# Combining all the above stundents 
from tqdm import tqdm
preprocessed_reviews = []
# tqdm is for printing the status bar
for sentance in tqdm(final['Text'].values):
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    # https://gist.github.com/sebleier/554280
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_reviews.append(sentance.strip())

In [ ]:
preprocessed_reviews[1500]

# [4] Featurization

# [4.1] Bag of Words (BoW)

In [ ]:
# Sparse Matrix : Matrix which contains very few non-zero elements
# We can represent Sparse Matrix in dictionary format instead of regular matrix representation to improve space complexity. => {(row,column): non-zero value}

count_vect = CountVectorizer() #in scikit-learn
count_vect.fit(preprocessed_reviews)
print("some feature names ", count_vect.get_feature_names()[:10])
print('='*50)

final_counts = count_vect.transform(preprocessed_reviews)
print("the type of count vectorizer ",type(final_counts))
print("the shape of out text BOW vectorizer ",final_counts.get_shape())
print("the number of unique words ", final_counts.get_shape()[1])

# [4.2] Bi-Grams and n-Grams

In [ ]:
#bi-gram, tri-gram and n-gram
#removing stop words like "not" should be avoided before building n-grams
# count_vect = CountVectorizer(ngram_range=(1,2))
# please do read the CountVectorizer documentation http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
# you can choose these numebrs min_df=10, max_features=5000, of your choice
count_vect = CountVectorizer(ngram_range=(1,2), min_df=10, max_features=5000)
final_bigram_counts = count_vect.fit_transform(preprocessed_reviews)
print("the type of count vectorizer ",type(final_bigram_counts))
print("the shape of out text BOW vectorizer ",final_bigram_counts.get_shape())
print("the number of unique words including both unigrams and bigrams ", final_bigram_counts.get_shape()[1])

# [4.3] TF-IDF

In [ ]:
tf_idf_vect = TfidfVectorizer(ngram_range=(1,2), min_df=10)
tf_idf_vect.fit(preprocessed_reviews)
print("some sample features(unique words in the corpus)",tf_idf_vect.get_feature_names()[0:10])
print('='*50)

final_tf_idf = tf_idf_vect.transform(preprocessed_reviews)
print("the type of count vectorizer ",type(final_tf_idf))
print("the shape of out text TFIDF vectorizer ",final_tf_idf.get_shape())
print("the number of unique words including both unigrams and bigrams ", final_tf_idf.get_shape()[1])

# [4.4] Word2Vec(W2V)

In [ ]:
# Train your own Word2Vec model using your own text corpus
i=0
list_of_sentance=[]
for sentance in preprocessed_reviews:
    list_of_sentance.append(sentance.split())

In [ ]:
# Using Google News Word2Vectors

# in this project we are using a pretrained model by google
# its 3.3G file, once you load this into your memory 
# it occupies ~9Gb, so please do this step only if you have >12G of ram
# we will provide a pickle file wich contains a dict , 
# and it contains all our courpus words as keys and  model[word] as values
# To use this code-snippet, download "GoogleNews-vectors-negative300.bin" 
# from https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit
# it's 1.9GB in size.


# http://kavita-ganesan.com/gensim-word2vec-tutorial-starter-code/#.W17SRFAzZPY
# you can comment this whole cell
# or change these varible according to your need

is_your_ram_gt_16g=False
want_to_use_google_w2v = False
want_to_train_w2v = True

if want_to_train_w2v:
    # min_count = 5 considers only words that occured atleast 5 times
    w2v_model=Word2Vec(list_of_sentance,min_count=5,size=50, workers=4)
    print(w2v_model.wv.most_similar('great'))
    print('='*50)
    print(w2v_model.wv.most_similar('worst'))
    
elif want_to_use_google_w2v and is_your_ram_gt_16g:
    if os.path.isfile('GoogleNews-vectors-negative300.bin'):
        w2v_model=KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
        print(w2v_model.wv.most_similar('great'))
        print(w2v_model.wv.most_similar('worst'))
    else:
        print("you don't have gogole's word2vec file, keep want_to_train_w2v = True, to train your own w2v ")

In [ ]:
w2v_words = list(w2v_model.wv.vocab)
print("number of words that occured minimum 5 times ",len(w2v_words))
print("sample words ", w2v_words[0:50])

# [4.4.1] Converting text(document) into Vectors using Avg W2V , TFIDF-W2V

[4.4.1.1] Avg W2v

In [ ]:
# average Word2Vec
# compute average word2vec for each review.
sent_vectors = []; # the avg-w2v for each sentence/review is stored in this list
for sent in tqdm(list_of_sentance): # for each review/sentence
    sent_vec = np.zeros(50) # as word vectors are of zero length 50, you might need to change this to 300 if you use google's w2v
    cnt_words =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        if word in w2v_words:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
    if cnt_words != 0:
        sent_vec /= cnt_words
    sent_vectors.append(sent_vec)
print(len(sent_vectors))
print(len(sent_vectors[0]))

[4.4.1.2] TFIDF weighted W2V

In [ ]:
# S = ["abc def pqr", "def def def abc", "pqr pqr def"]
model = TfidfVectorizer()
model.fit(preprocessed_reviews)
# we are converting a dictionary with word as a key, and the idf as a value
dictionary = dict(zip(model.get_feature_names(), list(model.idf_)))

In [ ]:
# TF-IDF weighted Word2Vec
tfidf_feat = model.get_feature_names() # tfidf words/col-names
# final_tf_idf is the sparse matrix with row= sentence, col=word and cell_val = tfidf

tfidf_sent_vectors = []; # the tfidf-w2v for each sentence/review is stored in this list
row=0;
for sent in tqdm(list_of_sentance): # for each review/sentence 
    sent_vec = np.zeros(50) # as word vectors are of zero length
    weight_sum =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        if word in w2v_words and word in tfidf_feat:
            vec = w2v_model.wv[word]
#             tf_idf = tf_idf_matrix[row, tfidf_feat.index(word)]
            # to reduce the computation we are 
            # dictionary[word] = idf value of word in whole courpus
            # sent.count(word) = tf valeus of word in this review
            tf_idf = dictionary[word]*(sent.count(word)/len(sent))
            sent_vec += (vec * tf_idf)
            weight_sum += tf_idf
    if weight_sum != 0:
        sent_vec /= weight_sum
    tfidf_sent_vectors.append(sent_vec)
    row += 1

# [5] Applying TSNE

<ol> 
    <li> you need to plot 4 tsne plots with each of these feature set
        <ol>
            <li>Review text, preprocessed one converted into vectors using (BOW)</li>
            <li>Review text, preprocessed one converted into vectors using (TFIDF)</li>
            <li>Review text, preprocessed one converted into vectors using (AVG W2v)</li>
            <li>Review text, preprocessed one converted into vectors using (TFIDF W2v)</li>
        </ol>
    </li>
    <li> <font color='blue'>Note 1: The TSNE accepts only dense matrices</font></li>
    <li> <font color='blue'>Note 2: Consider only 5k to 6k data points </font></li>
</ol>

# [5.1] Applying TSNE on Text BOW Vectors

In [ ]:
# Case 1 : n_components=2, perplexity=30, n_iter=1000, random_state=0

#Applying t-SNE [ (d = 12997) ====> (d' = 2) ]
tsne = TSNE(n_components=2, perplexity=30, n_iter=1000, random_state=0)
x_bow_embedding = tsne.fit_transform(final_counts.todense()) # converting to dense matrix from sparse matrix because t-SNE takes dense matrix as input

# horizental stacking the features with lbls
tsne_on_bow = np.hstack((x_bow_embedding,np.array(final["Score"]).reshape(-1,1)))


#Creating dataframe
df_tsne_on_bow = pd.DataFrame(data=tsne_on_bow, columns=['Dimension_x','Dimension_y','lbl'])

# lbl : 0 -> bad && 1 -> good
df_tsne_on_bow["lbl"] = df_tsne_on_bow["lbl"].map(lambda x: "bad" if x == 0 else "good")


In [ ]:
#Scatter Plotting
sns.FacetGrid(df_tsne_on_bow , hue = "lbl", height = 4)\
    .map(plt.scatter, "Dimension_x" , "Dimension_y")\
    .add_legend()
plt.title("Scatter Plot || TSNE on Text BOW Vectors ||  Case 1 :  perplexity=30 and  n_iter=1000\n", fontdict={"fontsize" : 20})

In [ ]:
# Case 2 : n_components=2, perplexity=50, n_iter=2000, random_state=0

#Applying t-SNE [ (d = 12997) ====> (d' = 2) ]
tsne = TSNE(n_components=2, perplexity=50, n_iter=2000, random_state=0)
x_bow_embedding1 = tsne.fit_transform(final_counts.todense()) # converting to dense matrix from sparse matrix because t-SNE takes dense matrix as input

# horizental stacking the features with lbls
tsne_on_bow1 = np.hstack((x_bow_embedding1,np.array(final["Score"]).reshape(-1,1)))


#Creating dataframe
df_tsne_on_bow1 = pd.DataFrame(data=tsne_on_bow1, columns=['Dimension_x','Dimension_y','lbl'])

# lbl : 0 -> bad && 1 -> good
df_tsne_on_bow1["lbl"] = df_tsne_on_bow1["lbl"].map(lambda x: "bad" if x == 0 else "good")


In [ ]:
#Scatter Plotting
sns.FacetGrid(df_tsne_on_bow1 , hue = "lbl", height = 4)\
    .map(plt.scatter, "Dimension_x" , "Dimension_y")\
    .add_legend()
plt.title("Scatter Plot || TSNE on Text BOW Vectors ||  Case 2 :  perplexity=50 and  n_iter=2000\n", fontdict={"fontsize" : 20})

In [ ]:
# Case 3 : n_components=2, perplexity=50, n_iter=3000, random_state=0

#Applying t-SNE [ (d = 12997) ====> (d' = 2) ]
tsne = TSNE(n_components=2, perplexity=50, n_iter=3000, random_state=0)
x_bow_embedding2 = tsne.fit_transform(final_counts.todense()) # converting to dense matrix from sparse matrix because t-SNE takes dense matrix as input

# horizental stacking the features with lbls
tsne_on_bow2 = np.hstack((x_bow_embedding2,np.array(final["Score"]).reshape(-1,1)))


#Creating dataframe
df_tsne_on_bow2 = pd.DataFrame(data=tsne_on_bow2, columns=['Dimension_x','Dimension_y','lbl'])

# lbl : 0 -> bad && 1 -> good
df_tsne_on_bow2["lbl"] = df_tsne_on_bow2["lbl"].map(lambda x: "bad" if x == 0 else "good")


In [ ]:
#Scatter Plotting
sns.FacetGrid(df_tsne_on_bow2 , hue = "lbl", height = 4)\
    .map(plt.scatter, "Dimension_x" , "Dimension_y")\
    .add_legend()
plt.title("Scatter Plot || TSNE on Text BOW Vectors ||  Case 3 :  perplexity=50 and  n_iter=3000\n", fontdict={"fontsize" : 20})

# [5.2] Applying T-SNE on Text TFIDF Vectors

In [ ]:
# Case 1 : n_components=2, perplexity=30, n_iter=1000, random_state=0

#Applying t-SNE [ (d = 3144) ====> (d' = 2) ]
tsne = TSNE(n_components=2, perplexity=30, n_iter=1000, random_state=0)
x_tfidf_embedding = tsne.fit_transform(final_tf_idf.todense())

# horizental stacking the features with lbls
tsne_on_tfidf = np.hstack((x_tfidf_embedding,np.array(final["Score"]).reshape(-1,1)))


#Creating dataframe
df_tsne_on_tfidf = pd.DataFrame(data=tsne_on_tfidf, columns=['Dimension_x','Dimension_y','lbl'])

# lbl : 0 -> bad && 1 -> good
df_tsne_on_tfidf["lbl"] = df_tsne_on_tfidf["lbl"].map(lambda x: "bad" if x == 0 else "good")


In [ ]:
#Scatter Plotting
sns.FacetGrid(df_tsne_on_tfidf , hue = "lbl", height = 4)\
    .map(plt.scatter, "Dimension_x" , "Dimension_y")\
    .add_legend()
plt.title("Scatter Plot || TSNE on Text TF-IDF Vectors ||  Case 1 :  perplexity=30 and  n_iter=1000\n", fontdict={"fontsize" : 20})

In [ ]:
# Case 2 : n_components=2, perplexity=50, n_iter=2000, random_state=0

#Applying t-SNE [ (d = 3144) ====> (d' = 2) ]
tsne = TSNE(n_components=2, perplexity=50, n_iter=2000, random_state=0)
x_tfidf_embedding1 = tsne.fit_transform(final_tf_idf.todense())

# horizental stacking the features with lbls
tsne_on_tfidf1 = np.hstack((x_tfidf_embedding1,np.array(final["Score"]).reshape(-1,1)))


#Creating dataframe
df_tsne_on_tfidf1 = pd.DataFrame(data=tsne_on_tfidf1, columns=['Dimension_x','Dimension_y','lbl'])

# lbl : 0 -> bad && 1 -> good
df_tsne_on_tfidf1["lbl"] = df_tsne_on_tfidf1["lbl"].map(lambda x: "bad" if x == 0 else "good")


In [ ]:
#Scatter Plotting
sns.FacetGrid(df_tsne_on_tfidf1 , hue = "lbl", height = 4)\
    .map(plt.scatter, "Dimension_x" , "Dimension_y")\
    .add_legend()
plt.title("Scatter Plot || TSNE on Text TF-IDF Vectors ||  Case 2 :  perplexity=50 and  n_iter=2000\n", fontdict={"fontsize" : 20})

In [ ]:
# Case 3 : n_components=2, perplexity=50, n_iter=3000, random_state=0

#Applying t-SNE [ (d = 3144) ====> (d' = 2) ]
tsne = TSNE(n_components=2, perplexity=50, n_iter=3000, random_state=0)
x_tfidf_embedding2 = tsne.fit_transform(final_tf_idf.todense())

# horizental stacking the features with lbls
tsne_on_tfidf2 = np.hstack((x_tfidf_embedding2,np.array(final["Score"]).reshape(-1,1)))


#Creating dataframe
df_tsne_on_tfidf2 = pd.DataFrame(data=tsne_on_tfidf2, columns=['Dimension_x','Dimension_y','lbl'])

# lbl : 0 -> bad && 1 -> good
df_tsne_on_tfidf2["lbl"] = df_tsne_on_tfidf2["lbl"].map(lambda x: "bad" if x == 0 else "good")

In [ ]:
#Scatter Plotting
sns.FacetGrid(df_tsne_on_tfidf2 , hue = "lbl", height = 4)\
    .map(plt.scatter, "Dimension_x" , "Dimension_y")\
    .add_legend()
plt.title("Scatter Plot || TSNE on Text TF-IDF Vectors ||  Case 3 :  perplexity=50 and  n_iter=3000\n", fontdict={"fontsize" : 20})

# [5.3] Applying T-SNE on Text Avg-W2V Vectors

In [ ]:
# Case 1 : n_components=2, perplexity=30, n_iter=1000, random_state=0

#Applying t-SNE [ (d = 50) ====> (d' = 2) ]
tsne = TSNE(n_components=2, perplexity=30, n_iter=1000, random_state=0)
x_avg_w2v_embedding = tsne.fit_transform(np.matrix(sent_vectors)) # converting 2-d list to numpy matrix

# horizental stacking the features with lbls
tsne_on_avg_w2v = np.hstack((x_avg_w2v_embedding, np.array(final["Score"]).reshape(-1,1)))


#Creating dataframe
df_tsne_on_avg_w2v = pd.DataFrame(data=tsne_on_avg_w2v, columns=['Dimension_x','Dimension_y','lbl'])

# lbl : 0 -> bad && 1 -> good
df_tsne_on_avg_w2v["lbl"] = df_tsne_on_avg_w2v["lbl"].map(lambda x: "bad" if x == 0 else "good")


In [ ]:
#Scatter Plotting
sns.FacetGrid(df_tsne_on_avg_w2v , hue = "lbl", height = 4)\
    .map(plt.scatter, "Dimension_x" , "Dimension_y")\
    .add_legend()
plt.title("Scatter Plot || TSNE on Text Avg-W2V Vectors ||  Case 1 :  perplexity=30 and  n_iter=1000\n", fontdict={"fontsize" : 20})

In [ ]:
# Case 2 : n_components=2, perplexity=50, n_iter=2000, random_state=0

#Applying t-SNE [ (d = 50) ====> (d' = 2) ]
tsne = TSNE(n_components=2, perplexity=50, n_iter=2000, random_state=0)
x_avg_w2v_embedding1 = tsne.fit_transform(np.matrix(sent_vectors))

# horizental stacking the features with lbls
tsne_on_avg_w2v1 = np.hstack((x_avg_w2v_embedding1, np.array(final["Score"]).reshape(-1,1)))


#Creating dataframe
df_tsne_on_avg_w2v1 = pd.DataFrame(data=tsne_on_avg_w2v1, columns=['Dimension_x','Dimension_y','lbl'])

# lbl : 0 -> bad && 1 -> good
df_tsne_on_avg_w2v1["lbl"] = df_tsne_on_avg_w2v1["lbl"].map(lambda x: "bad" if x == 0 else "good")


In [ ]:
#Scatter Plotting
sns.FacetGrid(df_tsne_on_avg_w2v1 , hue = "lbl", height = 4)\
    .map(plt.scatter, "Dimension_x" , "Dimension_y")\
    .add_legend()
plt.title("Scatter Plot || TSNE on Text Avg-W2V Vectors ||  Case 2 :  perplexity=50 and  n_iter=2000\n", fontdict={"fontsize" : 20})

In [ ]:
# Case 3 : n_components=2, perplexity=50, n_iter=3000, random_state=0

#Applying t-SNE [ (d = 50) ====> (d' = 2) ]
tsne = TSNE(n_components=2, perplexity=50, n_iter=3000, random_state=0)
x_avg_w2v_embedding2 = tsne.fit_transform(np.matrix(sent_vectors))

# horizental stacking the features with lbls
tsne_on_avg_w2v2 = np.hstack((x_avg_w2v_embedding2, np.array(final["Score"]).reshape(-1,1)))


#Creating dataframe
df_tsne_on_avg_w2v2 = pd.DataFrame(data=tsne_on_avg_w2v2, columns=['Dimension_x','Dimension_y','lbl'])

# lbl : 0 -> bad && 1 -> good
df_tsne_on_avg_w2v2["lbl"] = df_tsne_on_avg_w2v2["lbl"].map(lambda x: "bad" if x == 0 else "good")


In [ ]:
#Scatter Plotting
sns.FacetGrid(df_tsne_on_avg_w2v2 , hue = "lbl", height = 4)\
    .map(plt.scatter, "Dimension_x" , "Dimension_y")\
    .add_legend()
plt.title("Scatter Plot || TSNE on Text Avg-W2V Vectors ||  Case 3 :  perplexity=50 and  n_iter=3000\n", fontdict={"fontsize" : 20})

# [5.4] Applying TSNE on Text TFIDF-weighted W2V Vectors

In [ ]:
# Case 1: n_components=2, perplexity=30, n_iter=1000, random_state=0

#Applying t-SNE [ (d = 50) ====> (d' = 2) ]
tsne = TSNE(n_components=2, perplexity=30, n_iter=1000, random_state=0)
x_tfidf_w2v_embedding = tsne.fit_transform(np.matrix(tfidf_sent_vectors)) # Converting 2-d list to numpy matrix

# horizental stacking the features with lbls
tsne_on_tfidf_w2v = np.hstack((x_tfidf_w2v_embedding, np.array(final["Score"]).reshape(-1,1)))


#Creating dataframe
df_tsne_on_tfidf_w2v = pd.DataFrame(data=tsne_on_tfidf_w2v, columns=['Dimension_x','Dimension_y','lbl'])

# lbl : 0 -> bad && 1 -> good
df_tsne_on_tfidf_w2v["lbl"] = df_tsne_on_tfidf_w2v["lbl"].map(lambda x: "bad" if x == 0 else "good")


In [ ]:
#Scatter Plotting
sns.FacetGrid(df_tsne_on_tfidf_w2v , hue = "lbl", height = 4)\
    .map(plt.scatter, "Dimension_x" , "Dimension_y")\
    .add_legend()
plt.title("Scatter Plot || TSNE on Text tfidf-Weighted W2V Vectors ||  Case 1 :  perplexity=30 and  n_iter=1000\n", fontdict={"fontsize" : 20})

In [ ]:
# Case 2: n_components=2, perplexity=50, n_iter=2000, random_state=0

#Applying t-SNE [ (d = 50) ====> (d' = 2) ]
tsne = TSNE(n_components=2, perplexity=50, n_iter=2000, random_state=0)
x_tfidf_w2v_embedding1 = tsne.fit_transform(np.matrix(tfidf_sent_vectors))

# horizental stacking the features with lbls
tsne_on_tfidf_w2v1 = np.hstack((x_tfidf_w2v_embedding1, np.array(final["Score"]).reshape(-1,1)))


#Creating dataframe
df_tsne_on_tfidf_w2v1 = pd.DataFrame(data=tsne_on_tfidf_w2v1, columns=['Dimension_x','Dimension_y','lbl'])

# lbl : 0 -> bad && 1 -> good
df_tsne_on_tfidf_w2v1["lbl"] = df_tsne_on_tfidf_w2v1["lbl"].map(lambda x: "bad" if x == 0 else "good")


In [ ]:
#Scatter Plotting
sns.FacetGrid(df_tsne_on_tfidf_w2v1 , hue = "lbl", height = 4)\
    .map(plt.scatter, "Dimension_x" , "Dimension_y")\
    .add_legend()
plt.title("Scatter Plot || TSNE on Text tfidf-Weighted W2V Vectors ||  Case 2 :  perplexity=50 and  n_iter=2000\n", fontdict={"fontsize" : 20})

In [ ]:
# Case 3: n_components=2, perplexity=50, n_iter=3000, random_state=0

#Applying t-SNE [ (d = 50) ====> (d' = 2) ]
tsne = TSNE(n_components=2, perplexity=50, n_iter=3000, random_state=0)
x_tfidf_w2v_embedding2 = tsne.fit_transform(np.matrix(tfidf_sent_vectors))

# horizental stacking the features with lbls
tsne_on_tfidf_w2v2 = np.hstack((x_tfidf_w2v_embedding2, np.array(final["Score"]).reshape(-1,1)))


#Creating dataframe
df_tsne_on_tfidf_w2v2 = pd.DataFrame(data=tsne_on_tfidf_w2v2, columns=['Dimension_x','Dimension_y','lbl'])

# lbl : 0 -> bad && 1 -> good
df_tsne_on_tfidf_w2v2["lbl"] = df_tsne_on_tfidf_w2v2["lbl"].map(lambda x: "bad" if x == 0 else "good")


In [ ]:
#Scatter Plotting
sns.FacetGrid(df_tsne_on_tfidf_w2v2 , hue = "lbl", height = 4)\
    .map(plt.scatter, "Dimension_x" , "Dimension_y")\
    .add_legend()
plt.title("Scatter Plot || TSNE on Text tfidf-Weighted W2V Vectors ||  Case 3 :  perplexity=50 and  n_iter=3000\n", fontdict={"fontsize" : 20})